In [106]:

import pandas as pd 
import torch
import torch.nn as nn 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import torchvision
import pandas as pd
import numpy as np
import torch.nn.functional as F 
from sklearn.preprocessing import StandardScaler
import json 

In [107]:
df = pd.read_csv('CCFC_match_lineups_data.csv')
df.head()

,Unnamed: 0,Opposition,Distance,HSR,Sprint,Accelerations,Decelerations,Jumps,match_id,season_name,...,final_third_possession,ppda,completed_passes_and_carries_into_final_third,match_outcome,date,monte_carlo_win_prob,monte_carlo_draw_prob,monte_carlo_loss_prob,location,lineup
0,0,MK Dons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,...,NaN,NaN,NaN,NaN,2020-09-05,NaN,NaN,NaN,away,[]
1,1,Bristol,110778.68,7095.44,1315.26,280.0,405.0,50.0,3764253.0,2021,...,58.0,9.285714,43.0,lost,2020-09-12,0.2878,0.2962,0.4160,away,"[{'player_id': 3355, 'squad_role': 'Starting X..."
2,2,Gillingham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,...,NaN,NaN,NaN,NaN,2020-09-15,NaN,NaN,NaN,away,[]
3,3,QPR,111807.25,7601.80,1511.42,293.0,404.0,33.0,3764258.0,2021,...,41.0,20.833333,38.0,won,2020-09-18,0.3171,0.3402,0.3427,home,"[{'player_id': 3355, 'squad_role': 'Starting X..."
4,4,Barnsley,106883.57,7861.21,1346.77,323.0,454.0,36.0,3766905.0,2021,...,45.0,7.500000,35.0,draw,2020-09-26,0.2182,0.3205,0.4613,away,"[{'player_id': 3355, 'squad_role': 'Starting X..."


In [108]:
def preprocess_data(df):
    original_shape = df.shape
    df_cleaned = df.copy()
    df_cleaned = df_cleaned.dropna(how='any') # drop empty columns

    numeric_columns = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    df_cleaned[numeric_columns] = scaler.fit_transform(df_cleaned[numeric_columns])

    if 'result' in df_cleaned.columns:
        df_cleaned['result'] = (df_cleaned['result'] == 'win').astype(int)

    return df_cleaned

In [109]:
df = preprocess_data(df)


In [119]:
import ast 
# location
# Opposition


players = [] # stores all the different players there are. 
squads = []
homes = [] # 1 = home, 0 = away 
oppos_index = []
wins = []

for index, row in df.iterrows():
    # player stuff
    lineup = row['lineup']
    data = ast.literal_eval(lineup)
    squad = []
    for squad_player in data: 
        if squad_player['squad_role'] == 'Starting XI':
            squad.append(squad_player['player_id'])
            if squad_player['player_id'] not in players: 
                players.append(squad_player['player_id'])
    squads.append(squad)

    # home/ away stufuf
    if row['location'] == "away": homes.append(0)
    elif row['location'] == "home": homes.append(1)
    else: print('uh oh spagettio')

    # team stuff 
    oppos_index.append(row['opposition_team'])

    if row['match_outcome'] == 'lost': wins.append(0)
    elif row['match_outcome'] == 'draw': wins.append(1)
    elif row['match_outcome'] == 'won': wins.append(2)
    else: print('uhhhhh oh spagettio')


players.sort()
oppo = df['opposition_team'].unique()
diff_oppos = df['opposition_team'].unique()
diff_oppos.sort()
print(diff_oppos)
print(len(oppo))
x_2 = np.zeros(len(oppo))
print(len(diff_oppos))

# print(len(players))
# print(players[2])


# # this is for 1 game 
# # 52 players + home/ away + team
# x = np.zeros(52)
# player_ids = [7847, 3317]
# # 52 different players, home and away
# for id in player_ids: 
#     x[players.index(id)] = 1

# print(x)



['AFC Bournemouth' 'Barnsley' 'Birmingham City' 'Blackburn Rovers'
 'Blackpool' 'Brentford' 'Bristol City' 'Burnley' 'Cardiff City'
 'Derby County' 'Fulham' 'Huddersfield Town' 'Hull City' 'Ipswich Town'
 'Leeds United' 'Leicester City' 'Luton Town' 'Middlesbrough' 'Millwall'
 'Norwich City' 'Nottingham Forest' 'Peterborough United'
 'Plymouth Argyle' 'Preston North End' 'Queens Park Rangers' 'Reading'
 'Rotherham United' 'Sheffield United' 'Sheffield Wednesday' 'Southampton'
 'Stoke City' 'Sunderland' 'Swansea City' 'Watford' 'West Bromwich Albion'
 'Wigan Athletic' 'Wycombe Wanderers']
37
37


In [111]:
df['match_outcome'].unique()

array(['lost', 'won', 'draw'], dtype=object)

In [112]:
#[loss, draw, win ]
Y = []
for win in wins: 
    array = np.zeros(3) 
    if win == 0: array[0] = 1 
    elif win == 1: array[1] = 1 
    elif win == 2: array[2] = 1 
    Y.append(array)


In [113]:
print(len(squads))
print(len(diff_oppos))
print(len(players))
print(len(homes))

X = []


for i in range(len(squads)):
    x_players = np.zeros(len(players))
    x_oppos = np.zeros(len(diff_oppos))
    starting_11, oppo, home = squads[i], oppos_index[i], homes[i]
    for player_id in starting_11: 
        x_players[players.index(player_id)] = 1 
    x_oppos[diff_oppos.tolist().index(oppo)] = 1 
    X.append(np.concatenate([x_players, x_oppos, np.array([home])]))
    
print(len(X))
print(len(Y))


187
37
52
187
187
187


In [114]:
print(X)

[array([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.]), array([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0.

In [115]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Set device
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

# X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
# Y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)

# # Define the model
# class SimpleModel(nn.Module):
#     def __init__(self, input_size=90, hidden_size=128, output_size=3):
#         super(SimpleModel, self).__init__()
        
#         # Define layers
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
#         self.fc3 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.fc3(x)
#         return x

# # Initialize model, loss, and optimizer
# model = SimpleModel().to(device)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# epochs = 20 # 20 
# batch_size = 11
# num_batches = len(X_tensor) // batch_size

# for epoch in range(epochs):
#     for i in range(num_batches):
#         # Get the batch of inputs and labels
#         start = i * batch_size
#         end = start + batch_size
#         inputs = X_tensor[start:end]   # Shape: (batch_size, 240)
#         labels = Y_tensor[start:end]   # Shape: (batch_size, 3)

#         # Forward pass
#         outputs = model(inputs)        # Expected shape: (batch_size, 3)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [116]:
# # Define the path to save the model
# model_path = "./Models/first_model.pth"

# # Save the model's state_dict and optimizer's state_dict
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict()
# }, model_path)

# print(f"Model saved to {model_path}")

In [117]:
print(diff_oppos)

['AFC Bournemouth' 'Barnsley' 'Birmingham City' 'Blackburn Rovers'
 'Blackpool' 'Brentford' 'Bristol City' 'Burnley' 'Cardiff City'
 'Derby County' 'Fulham' 'Huddersfield Town' 'Hull City' 'Ipswich Town'
 'Leeds United' 'Leicester City' 'Luton Town' 'Middlesbrough' 'Millwall'
 'Norwich City' 'Nottingham Forest' 'Peterborough United'
 'Plymouth Argyle' 'Preston North End' 'Queens Park Rangers' 'Reading'
 'Rotherham United' 'Sheffield United' 'Sheffield Wednesday' 'Southampton'
 'Stoke City' 'Sunderland' 'Swansea City' 'Watford' 'West Bromwich Albion'
 'Wigan Athletic' 'Wycombe Wanderers']
